# HackerNews Dataset Analysis
This dataset contains all stories and comments from [HackerNews](https://news.ycombinator.com/news)  from its launch in 2006. Each story contains a story id, the author that made the post, when it was written, and the number of points the story received. <br>
Hacker News is a social news website focusing on computer science and entrepreneurship. It is run by Paul Graham's investment fund and startup incubator, Y Combinator. In general, content that can be submitted is defined as "anything that gratifies one's intellectual curiosity". <br>

There are 4 tables in this public dataset:
- stories (a.k.a posts)
- comments
- full
- full_2015 <br>
<br>
* An example [post](https://news.ycombinator.com/item?id=8596682) from HackerNews.
* An example of a [comment](https://news.ycombinator.com/item?id=8597333) from HackerNews.
* A post can have many comments.
* A comment can have many comments.
* Posts can have scores.
* Posts have rankings showing their popularity.


So the structure is:
* Story
  * comment
    * comment
      * comment

**Now the question is, how this structure represented in the tables ?**

# 1) Understanding the structure between *stories* and *comments* tables
Before diving deeper in the queries lets understand the structure of the tables by investigating following questions:
* Parent_id column in the comments table: 
  * what do they represent? 
  * do they only represent parent id of the comment or the parent id of the story?
* How much of a comments does a story has, which column represents it?
* Are the comment and story ids globally unique?

In [1]:
import os
import pandas as pd
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Stories Table - First Rows

In [2]:
# view first rows of stories table
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# Construct a reference to the "stories" table
table_ref_stories = dataset_ref.table("stories")

# API request - fetch the table
table_stories = client.get_table(table_ref_stories)

# Preview the first five lines of the "stories" table
client.list_rows(table_stories, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,id,by,score,time,time_ts,title,url,text,deleted,dead,descendants,author
0,6940813,sarath237,0,1387536270,2013-12-20 10:44:30+00:00,Sheryl Brindo Hot Pics,http://www.youtube.com/watch?v=ym1cyxneB0Y,Sheryl Brindo Hot Pics,None,True,NaN,sarath237
1,6991401,123123321321,0,1388508751,2013-12-31 16:52:31+00:00,Are you people also put off by the culture of ...,,They&#x27;re pretty explicitly &#x27;startup f...,None,True,NaN,123123321321
2,1531556,ssn,0,1279617234,2010-07-20 09:13:54+00:00,New UI for Google Image Search,http://googlesystem.blogspot.com/2010/07/googl...,Again following on Bing's lead.,None,None,0.0,ssn
3,5012398,hoju,0,1357387877,2013-01-05 12:11:17+00:00,Historic website screenshots,http://webscraping.com/blog/Generate-website-s...,Python script to generate historic screenshots...,None,None,0.0,hoju
4,7214182,kogir,0,1401561740,2014-05-31 18:42:20+00:00,Placeholder,,Mind the gap.,None,None,0.0,kogir


**Stories**
* `id` is the unique key of the story.
* `descendants` is the number of all comments that a story has.
* Points of a story is represented as score in the table.

## Stories Table - Attributes and Field Data Types

In [3]:
table_stories.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique story ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of submitter', ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story text', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story descendants', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ())]

## Comments Table - First Rows

In [4]:
# Construct a reference to the "comments" table
table_ref_comments = dataset_ref.table("comments")

# API request - fetch the table
table_comments = client.get_table(table_ref_comments)

# Preview the first five lines of the "comments" table
client.list_rows(table_comments, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


## Comments Table - Attributes and Field Data Types

In [5]:
table_comments.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique comment ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of commenter', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ())]

## Parent column in the `comments` table, what does it represent?

In [6]:
# look at the parent_id column in the comments table
# then search for parent_ids who does not represent a comment_id in the comments table
# if this results are no empty, it means parent column represent parend comment id and one another id
query_for_id_selection = """ 
                            SELECT DISTINCT(parent)
                            FROM `bigquery-public-data.hacker_news.comments`
                            WHERE parent NOT IN (
                                                    SELECT id
                                                    FROM `bigquery-public-data.hacker_news.comments`)
            
"""
# set quota not to exceed limits
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

# create job to execute query
query_for_id_selection_job = client.query(query_for_id_selection, job_config=safe_config)

#load results into a dataframe
query_for_id_result = query_for_id_selection_job.to_dataframe()

# turn dataframe to a a series, then list, then list of strings to use in the query
non_comment_ids = ",".join(map(str,query_for_id_result.head(20).parent.tolist()))

In [7]:
# function to write query results to a dataframe
# without exceeding the 1 GB quota per query
def query_to_dataframe(query_name):
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    query_name_job = client.query(query_name, job_config=safe_config)
    return query_name_job.to_dataframe()

In [8]:
print("Number of parent_ids that are not comment_ids:", query_for_id_result.parent.nunique())

Number of parent_ids that are not comment_ids: 485055


**Conclusion 1:** There are 485055 parent_ids that does not represent to a comment_id. Lets have a look at some of them.

In [9]:
non_comment_ids

'7868175,1155342,8703793,3085102,3917074,4318042,2777883,4727721,165587,1771361,1139768,4684420,1340396,8411638,2684620,6495507,4625764,5483767,10097630,7081246'

In [10]:
# are parent_ids that does not represent a comment_id, do they belong to a story?
non_comment_id_query = """ 
                            SELECT *
                            FROM `bigquery-public-data.hacker_news.stories` AS stories
                            WHERE id IN ({})
                    """.format(non_comment_ids)

# print query results
query_to_dataframe(non_comment_id_query).head()

,id,by,score,time,time_ts,title,url,text,deleted,dead,descendants,author
0,3917074,ahsanhilal,34,1335919322,2012-05-02 00:42:02+00:00,Ask HN: Socialcam spamming Facebook newsfeeds ...,,So I clicked play on a social cam video somebo...,None,None,4,ahsanhilal
1,4625764,zengr,2,1349678518,2012-10-08 06:41:58+00:00,Einstein letter on God to be auctioned on eBay,http://www.huffingtonpost.com/huff-wires/20121...,,None,None,1,zengr
2,4318042,patrickod,273,1343751808,2012-07-31 16:23:28+00:00,How I made $10k in one day with Facebook Ads,http://irvinebroque.tumblr.com/post/2841539387...,,None,None,113,patrickod
3,165587,NickSmith,2,1208377299,2008-04-16 20:21:39+00:00,Aha! moments in Lisp... What were your 'Aha mo...,http://groups.google.com/group/comp.lang.lisp/...,,None,None,3,NickSmith
4,5483767,markchristian,81,1364955284,2013-04-03 02:14:44+00:00,Show HN: my new app Timebar has a clever UI (I...,,"Howdy, gang;\nAfter almost two weeks of sittin...",None,None,56,markchristian


**Conclusion 2:** Some of the non_comment_ids are story_ids.

## How much of comments does a story has, which column represents it?

In [11]:
# look at parent id row 4318042 to see how many comments are dependant on it
# in the stories table, number of  descendants are 113
query_number_of_comments = """
                            SELECT *
                            FROM `bigquery-public-data.hacker_news.comments`
                            WHERE parent = 4318042
"""

print("Number of comments that story 4318042 has:", len(query_to_dataframe(query_number_of_comments)))
# it did not give 122 rows because of the tree structure.
# it only give the first level comment id which is 12.

Number of comments that story 4318042 has: 53


* When number of descendants were 113, number of comments belong to that story is 53.
* This is an expected result because of the tree structure that post and comments have.
* 53 is the number of first level comments created for that story, 113 is the number of all sub-comments that a story has. <br>

**Conclusion 3:** Descendants attribute are the total number of comments including sub-comments that a story has.

## Are id columns in the stories and comments globally unique ?

In [12]:
id_query = """ 
                SELECT id
                FROM `bigquery-public-data.hacker_news.stories`
                WHERE id IN (
                            SELECT id
                            FROM `bigquery-public-data.hacker_news.comments`
                            )
"""

query_to_dataframe(id_query)

,id


**Conclusion 4:** Ids are globally unique among stories and comments table.

# 2) With the conclusions in mind, some interesting questions to ask:
* Recent studies have found that many forums tend to be dominated by a very small fraction of users. Is this true of Hacker News?
* Hacker News has received complaints that the site is biased towards Y Combinator startups. Do the data support this?
* What is the average number of daily comments created per day?
* What is the number of users that HackerNews had over years?
* How long does it take for a post to receive comment?
* How many of the comments receive sub-comments ?
* Is it more common for users to first create post or provide comments?
* For the users who joined the site in January 2014. When did they post their first story or comment, if ever?
* How many distinct users posted on October, 2015?
* What is the moving average (within the 15 day window) of number of posts created in each post category? 
* What is the rank of the stories based on scores, created in the same day ?


## a) Recent studies have found that many forums tend to be dominated by a very small fraction of users. Is this true of Hacker News?
* number of posts created
* number of comments created

shows a sign of being a active and dominated user in the HackerNews.


In [13]:
# to have a look we are going to count the number of comments and posts created 
# for not deleted and dead ones
active_users_query = """
                        WITH active_users_from_stories AS (
                            SELECT author,
                                COUNT(*) AS number_of_stories
                            FROM `bigquery-public-data.hacker_news.stories`
                            WHERE deleted IS NOT TRUE AND dead IS NOT TRUE
                            GROUP BY author
                            ORDER BY number_of_stories DESC
                        ),
                        active_users_from_comments AS (
                            SELECT author,
                                COUNT(*) AS number_of_comments
                            FROM `bigquery-public-data.hacker_news.comments`
                            WHERE deleted IS NOT TRUE AND dead IS NOT TRUE
                            GROUP BY author
                            ORDER BY number_of_comments DESC
                        )
                        SELECT active_users_from_comments.author,
                            number_of_stories,
                            number_of_comments
                        FROM active_users_from_stories
                        FULL JOIN active_users_from_comments
                         ON active_users_from_stories.author = active_users_from_comments.author
                        ORDER BY number_of_stories DESC
                        LIMIT 10
"""
# if you change last ORDER BY clause to number_of_comments will list the users who commented most
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
active_users_query_job = client.query(active_users_query, job_config=safe_config)
active_users_query_result = active_users_query_job.to_dataframe()
active_users_query_result

,author,number_of_stories,number_of_comments
0,None,25721,NaN
1,cwan,7001,389.0
2,shawndumas,6507,967.0
3,evo_9,5456,506.0
4,nickb,4276,1152.0
5,iProject,4237,36.0
6,bootload,4151,2678.0
7,edw519,3788,4570.0
8,ColinWright,3702,5309.0
9,tokenadult,3622,6706.0


In [14]:
# look at the total number of stories in the whole hackernews which is not dead or deleted
total_stories = """
                    SELECT COUNT(id)
                    FROM `bigquery-public-data.hacker_news.stories`
                    WHERE deleted IS NOT TRUE AND dead IS NOT TRUE
"""
total_stories_df = client.query(total_stories).result().to_dataframe()

percentage = active_users_query_result.number_of_stories.sum()/total_stories_df.f0_.iloc[0]

print("Most active users in terms of number of stories created have created the {} of the whole stories"
      .format(round(percentage,2)))

Most active users in terms of number of stories created have created the 0.05 of the whole stories


## b) Hacker News has received complaints that the site is biased towards Y Combinator startups. Do the data support this?

Here are some of the most popular YCombinator startups:

* AirBnB
* Stripe
* Dropbox
* Zapier
* Reddit
and so on..

In [15]:
# before providing an answer to this question investigate and understand full table

# Construct a reference to the "full" table
table_ref_full = dataset_ref.table("full")

# API request - fetch the table
table_full = client.get_table(table_ref_full)

# Preview the first five lines of the "full" table
client.list_rows(table_full, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,&gt;Imagine being in a software team and disre...,None,nabla9,NaN,1508324981,2017-10-18 11:09:41+00:00,comment,15498153,15497888.0,NaN,None,None
1,"Need your opinion on PadKite, the first multit...",http://padkite.com,None,None,JoseVigil,40.0,1301020096,2011-03-25 02:28:16+00:00,story,2366954,NaN,34.0,None,None
2,None,None,70-80% of atmospheric oxygen comes from ocean ...,None,Nadya,NaN,1453402215,2016-01-21 18:50:15+00:00,comment,10947337,10946877.0,NaN,None,None
3,None,None,"<p><pre><code> OK, so what were they going to...",None,michaelt,NaN,1519121875,2018-02-20 10:17:55+00:00,comment,16419184,16419046.0,NaN,None,None
4,None,None,What surprised me was that ironically reductio...,None,salimmadjd,NaN,1535990892,2018-09-03 16:08:12+00:00,comment,17903321,17902533.0,NaN,None,None


In [16]:
# if those keywords occur in the full table's text or title column we can say they got covered
coverage_query = """WITH startup_ranking_score AS (
                    SELECT CASE 
                        WHEN title LIKE "%Airbnb%" OR text LIKE "%Airbnb%" THEN "Airbnb"
                        WHEN title LIKE "%Stripe%" OR text LIKE "%Stripe%" THEN "Stripe"
                        WHEN title LIKE "%Dropbox%" OR text LIKE "%Dropbox%" THEN "Dropbox"
                        WHEN title LIKE "%Zapier%" OR text LIKE "%Zapier%" THEN "Zapier"
                        WHEN title LIKE "%Reddit%" OR text LIKE "%Reddit%" THEN "Reddit"
                        END AS popular_startup_name,
                    ranking,
                    score
                    FROM `bigquery-public-data.hacker_news.full`
                                                                )
                    SELECT popular_startup_name,
                        SUM(ranking) AS total_ranking,
                        SUM(score) AS total_score
                    FROM startup_ranking_score
                    GROUP BY popular_startup_name
                    ORDER BY total_score DESC
"""
query_to_dataframe(coverage_query)

,popular_startup_name,total_ranking,total_score
0,None,None,38387600
1,Reddit,None,117769
2,Dropbox,None,89843
3,Stripe,None,57933
4,Airbnb,None,52437
5,Zapier,None,2778


YCombinator startups got covered by HackerNews. However, most of the stories belong to other subjects showing there is no bias towards those most popular startups.

## c) What is the average number of daily comments created per day?

In [17]:
# to look at this first total number of comments generated per day will be investigated
# then the average of the year will be aggregated
average_daily_comments_per_year = """ WITH total_comments_generated_per_day AS (
                                        SELECT EXTRACT(DAYOFYEAR FROM time_ts) AS day,
                                            EXTRACT(YEAR FROM time_ts) AS year,
                                            COUNT(id) AS total_comments
                                        FROM `bigquery-public-data.hacker_news.comments`
                                        GROUP BY year, day
                                        )
                                        SELECT year, 
                                            AVG(total_comments) AS average_daily_comments
                                        FROM total_comments_generated_per_day
                                        GROUP BY year
                                        ORDER BY year
"""
query_to_dataframe(average_daily_comments_per_year)

,year,average_daily_comments
0,2006,4.000000
1,2007,224.234177
2,2008,677.382514
3,2009,1345.435616
4,2010,2307.989041
5,2011,2862.682192
6,2012,3406.450820
7,2013,4562.136986
8,2014,4150.587912
9,2015,4473.744755


Looks like popularity of HackerNews increased constantly and dramatically from 2006 to 2015.

## d) What is the number of users that HackerNews had over years?

In [18]:
# this question will be investigated in the full table 
number_of_users = """
                    SELECT EXTRACT(YEAR FROM timestamp) AS year,
                        COUNT(DISTINCT f.by) AS number_of_users
                    FROM `bigquery-public-data.hacker_news.full` AS f
                    WHERE timestamp IS NOT NULL
                    GROUP BY year
                    ORDER BY year     
"""
query_to_dataframe(number_of_users)

,year,number_of_users
0,2006,16
1,2007,4158
2,2008,11936
3,2009,26497
4,2010,45217
5,2011,80301
6,2012,92339
7,2013,101526
8,2014,111154
9,2015,121848


Number of users constantly growed, supports the idea of the increasing popularity of HackerNews.

## e) How long does it take for a post to receive comment?

In [19]:
# to answer this question stories and comments tables will be joined and 
# and timedifference of the time_ts will be investigated
time_to_receive_comment = """
                            WITH time_difference AS (
                                SELECT stories.id AS story_id,
                                    MIN(TIMESTAMP_DIFF(comments.time_ts, stories.time_ts, SECOND)) AS second
                                FROM `bigquery-public-data.hacker_news.stories` AS stories
                                LEFT JOIN `bigquery-public-data.hacker_news.comments` AS comments
                                    ON stories.id = comments.parent
                                GROUP BY story_id
                                ORDER BY second ASC)
                            SELECT *
                            FROM time_difference
                            WHERE second >= 0
"""
query_to_dataframe(time_to_receive_comment).head(10)

,story_id,second
0,7674135,3
1,9429031,4
2,2533536,4
3,8137640,4
4,8101858,5
5,3460425,5
6,9645834,5
7,9150995,5
8,8285007,5
9,9043495,5


It takes only seconds a story to receive a comment.

In [20]:
print("Average number of hours passed for a story to receive a comment:",
      round(query_to_dataframe(time_to_receive_comment).second.mean()/3600,2))

Average number of hours passed for a story to receive a comment: 10.06


## f) How many of the comments receive sub-comments ?

In [21]:
# to answer this question we are going to look at the ratio of 
# comments receiving subcomments to all comments

# total number of comments in the comments table
total_num_comments = """
                        SELECT COUNT(DISTINCT id)
                        FROM `bigquery-public-data.hacker_news.comments`
                    """

# total number of comments having sub_comments in the comments table
total_num_comments_w_sub_comments = """ WITH comments_w_subcomment_list AS (
                                            SELECT id,
                                                CASE
                                                    WHEN id IN (
                                                        SELECT DISTINCT(parent)
                                                        FROM `bigquery-public-data.hacker_news.comments`) 
                                                            THEN 1
                                                    ELSE 0
                                                END AS is_commented
                                            FROM `bigquery-public-data.hacker_news.comments`)
                                        SELECT SUM(is_commented)
                                        FROM comments_w_subcomment_list
"""

In [22]:
percent_of_commented_comments = 100 * (query_to_dataframe(total_num_comments_w_sub_comments).f0_.iloc[0] 
                                      / query_to_dataframe(total_num_comments).f0_.iloc[0])

print("Percentage of comments with replies {}".format(round(percent_of_commented_comments,2)))

Percentage of comments with replies 43.82


## g) Is it more common for users to first create post or provide comments ?

In [23]:
# we are going to investigate this question in the union of comments and stories table
# with necessary attributes
user_and_creation_date = """ WITH authors_creation_times AS (
                                SELECT author, 
                                    time_ts,
                                        CASE
                                            WHEN author IS NOT NULL THEN "story"
                                                ELSE NULL
                                        END AS type
                                FROM `bigquery-public-data.hacker_news.stories` AS stories
                                UNION ALL
                                SELECT author,
                                    time_ts,
                                        CASE
                                            WHEN author IS NOT NULL THEN "comment"
                                                ELSE NULL
                                    END AS type
                                FROM `bigquery-public-data.hacker_news.comments` AS comments),
                            first_creation_date AS (
                                SELECT author,
                                    type,
                                    MIN(time_ts) AS first_activity_time
                                FROM authors_creation_times
                                GROUP BY author, type)
                            SELECT type,
                                COUNT(author) AS number_of_users
                            FROM first_creation_date
                            GROUP BY type        
"""
query_to_dataframe(user_and_creation_date)

,type,number_of_users
0,comment,236359
1,story,210368
2,None,0


People tend to comment first in the HackerNews. However, there is no significant difference between number of users first created a comment or story.

## h) For the users who joined the site in January 2014, When did they post their first story or comment, if ever?

In [24]:
# to answer this question
# first users who make their first activity on HackerNews on January 2014 will be identified
# then their activity will be matched from full table
# to make the query more efficient CTEs will be used rather than joining multiple tables at once

users_w_first_activity_2014_01 = """WITH users_from_2014_01 AS (
                                        SELECT f.by AS author,
                                            MIN(timestamp) AS first_activity
                                        FROM `bigquery-public-data.hacker_news.full` AS f
                                        WHERE timestamp >= '2014-01-01' AND timestamp < '2014-02-01'
                                        GROUP BY f.by)
                                    SELECT users_from_2014_01.author,
                                        users_from_2014_01.first_activity,
                                        f.type
                                    FROM users_from_2014_01
                                    LEFT JOIN `bigquery-public-data.hacker_news.full` AS f
                                    ON users_from_2014_01.author = f.by 
                                        AND users_from_2014_01.first_activity = f.timestamp   
                                """
query_to_dataframe(users_w_first_activity_2014_01).head(10)

,author,first_activity,type
0,jiyannwei,2014-01-08 07:26:19+00:00,job
1,kennystone,2014-01-22 14:43:06+00:00,job
2,nottombrown,2014-01-31 14:10:32+00:00,job
3,jbdowney,2014-01-16 07:09:50+00:00,job
4,harryzhang,2014-01-20 20:13:38+00:00,job
5,bradgessler,2014-01-09 07:33:02+00:00,job
6,kparashar,2014-01-15 04:59:20+00:00,job
7,None,2014-01-01 00:21:49+00:00,None
8,varunsrin,2014-01-29 20:34:15+00:00,job
9,srlake,2014-01-15 19:43:35+00:00,job


First activities of the users who discovered HackerNews on January 2914, is to create a job post or pollopt.

## i) How many distinct users posted on October, 2015?

In [25]:
# before answering this question lets look at the first rows of full_201510 table
# Construct a reference to the "full" table
table_ref_full = dataset_ref.table("full_201510")

# API request - fetch the table
table_full = client.get_table(table_ref_full)

# Preview the first five lines of the "full" table
client.list_rows(table_full, max_results=5).to_dataframe()

,by,score,time,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,danmaz74,None,1438616833,None,comment,None,Do those analyses also account for the energy ...,9996591,None,None,None,9996887,None
1,gbraad,None,1438616954,None,comment,None,Full stack engineer who&#x27;s proficient with...,9996335,None,None,None,9996912,None
2,qeorge,None,1260997308,None,comment,None,<i>It seems that IE has been a consistent impe...,999251,None,None,None,999709,None
3,Flammy,None,1438617965,None,comment,None,"Hey all,<p>Ivy Softworks is looking for talent...",9996333,None,None,None,9997096,None
4,scottliquid15,None,1438618127,None,comment,None,LiquidTalent is an exclusive marketplace for d...,9996333,None,None,None,9997137,None


In [26]:
# to answer this we are going to use full_201510 table
users_posted_201510 = """ 
                        SELECT COUNT(DISTINCT f.by) AS number_of_users
                        FROM `bigquery-public-data.hacker_news.full_201510` AS f
                     """
query_to_dataframe(users_posted_201510)

,number_of_users
0,352410


## j) What is the moving average (within the 15 day window) of number of posts created from 2018 and onwards, in each post category? 

In [27]:
# number of posts created temporary table will be created using full table
# and then moving averages per date and post category will be calculated
# using analytic functions
moving_average_query = """ WITH num_posts_per_day_type AS ( 
                            SELECT EXTRACT(DATE FROM timestamp) AS date,
                                type,
                                COUNT(id) AS num_posts
                            FROM `bigquery-public-data.hacker_news.full` 
                            WHERE timestamp >= "2018-01-01"
                            GROUP BY date, type
                            )
                          SELECT date,
                            type,
                            AVG(num_posts) OVER (
                                PARTITION BY type
                                ORDER BY num_posts
                                ROWS BETWEEN 7 PRECEDING AND 7 FOLLOWING) AS moving_average
                          FROM num_posts_per_day_type
                          ORDER BY date
"""
query_to_dataframe(moving_average_query).head(10)

,date,type,moving_average
0,2018-01-01,comment,4390.866667
1,2018-01-01,job,2.133333
2,2018-01-01,story,634.933333
3,2018-01-02,comment,7175.066667
4,2018-01-02,job,3.000000
5,2018-01-02,poll,1.000000
6,2018-01-02,pollopt,2.800000
7,2018-01-02,story,1134.933333
8,2018-01-03,comment,8425.866667
9,2018-01-03,job,7.000000


Number of posts created in story and comment category have a significantly higher moving averages than job, poll and pollopt category.

## k) What is the rank of the stories based on scores, created in the same day ?

In [28]:
# scores will be grouped and order per day created from scores table
# and a rank will be assigned using analytic functions
scores_query = """
                    SELECT id,
                        score,
                        EXTRACT(DATE FROM time_ts) AS date,
                        RANK() OVER(
                            PARTITION BY EXTRACT(DATE FROM time_ts)
                            ORDER BY score) AS score_rank
                    FROM `bigquery-public-data.hacker_news.stories`
                    WHERE score >= 0   
"""
query_to_dataframe(scores_query).head(10)

,id,score,date,score_rank
0,52,2,2006-10-12,1
1,51,3,2006-10-12,2
2,49,4,2006-10-12,3
3,50,4,2006-10-12,3
4,53,7,2006-10-12,5
5,48,9,2006-10-12,6
6,518,1,2007-02-22,1
7,523,1,2007-02-22,1
8,423,1,2007-02-22,1
9,527,1,2007-02-22,1
